## Integers in Python
In the old days of programming, computer memory was at a premium. Therefore, languages would give you pretty granular control over how many bytes to allocate for your data. Let’s take a quick peek at a few integer types from C as an example:


| Type  | Size    | Minimum Value  | Maximum Value |
| ---   | ------- | ---:           | ---:          |
| char	| 1 byte  | -128	       | 127           |
| short	| 2 bytes | -32,768	       | 32,767        |
| int	| 4 bytes | -2,147,483,648 | 2,147,483,647 |
| long	| 8 bytes | -9,223,372,036,854,775,808	| 9,223,372,036,854,775,807 |

These values might vary from platform to platform. However, such an abundance of numeric types allows you to arrange data in memory compactly. Remember that these don’t even include unsigned types!

On the other end of the spectrum are languages such as JavaScript, which have just one numeric type to rule them all. While this is less confusing for beginning programmers, it comes at the price of increased memory consumption, reduced processing efficiency, and decreased precision.

When talking about bitwise operators, it’s essential to understand how Python handles integer numbers. After all, you’ll use these operators mainly to work with integers. There are a couple of wildly different representations of integers in Python that depend on their values.

ref: https://realpython.com/python-bitwise-operators/#binary-number-representations

### Interned Integers : -5 ~ 256
In CPython, very small integers between -5 and 256 are interned in a global cache to gain some performance because numbers in that range are commonly used. In practice, whenever you refer to one of those values, which are singletons created at the interpreter startup, Python will always provide the same instance:

In [1]:
a = 256  # interned
b = 256  # interned
print(a is b)
print(id(a), id(b), sep="\n")

True
2238838546640
2238838546640


Both variables have the same identity because they refer to the exact same object in memory. That’s typical of reference types but not immutable values such as integers. However, when you go beyond that range of cached values, Python will start creating distinct copies during variable assignment:

In [1]:
a = 257
b = 257
print(a is b)
print(id(a), id(b), sep="\n")

False
4701715536
4701716368


Despite having equal values, these variables point to separate objects now. But don’t let that fool you. Python will occasionally jump in and optimize your code behind the scenes. For example, it’ll cache a number that occurs on the same line multiple times regardless of its value:

In [3]:
print(id(257), id(257), sep="\n")

1702141908400
1702141908400


Variables a and b are independent objects because they reside at different memory locations, while the numbers used literally in print() are, in fact, the same object.

Note: Interning is an implementation detail of the CPython interpreter, which might change in future versions, so don’t rely on it in your programs.

Interestingly, there’s a similar string interning mechanism in Python, which kicks in for short texts comprised of ASCII letters only. It helps speed up dictionary lookups by allowing their keys to be compared by memory addresses, or C pointers, instead of by the individual string characters.

### Fixed-Precision Integers
Integers that you’re most likely to find in Python will leverage the C signed long data type. They use the classic two’s complement binary representation on a fixed number of bits. The exact bit-length will depend on your hardware platform, operating system, and Python interpreter version.

Modern computers typically use 64-bit architecture, so this would translate to decimal numbers between -2^63 and 2^63 - 1. You can check the maximum value of a fixed-precision integer in Python in the following way:

In [2]:
import sys
print(sys.maxsize)
print(2**63-1)

9223372036854775807
9223372036854775807


It’s huge! Roughly 9 million times the number of stars in our galaxy, so it should suffice for everyday use. While the maximum value that you could squeeze out of the unsigned long type in C is even bigger, on the order of 1019, integers in Python have no theoretical limit. To allow this, numbers that don’t fit on a fixed-length bit sequence are stored differently in memory.

### Arbitrary-Precision Integers
Do you remember that popular K-pop song “Gangnam Style” that became a worldwide hit in 2012? The YouTube video was the first to break a billion views. Soon after that, so many people had watched the video that it made the view counter overflow. YouTube had no choice but to upgrade their counter from 32-bit signed integers to 64-bit ones.

That might give plenty of headroom for a view counter, but there are even bigger numbers that aren’t uncommon in real life, notably in the scientific world. Nonetheless, Python can deal with them effortlessly:

In [3]:
from math import factorial
f_42 = factorial(42)
print(f_42, f_42.bit_length())

1405006117752879898543142606244511569936384000000000 170


Since they’re well over the limits that any of the C types have to offer, such astronomical numbers are converted into a sign-magnitude positional system, whose base is 230. Yes, you read that correctly. Whereas you have ten fingers, Python has over a billion!

Again, this may vary depending on the platform you’re currently using. When in doubt, you can double-check:

In [8]:
import sys
sys.int_info

sys.int_info(bits_per_digit=30, sizeof_digit=4, default_max_str_digits=4300, str_digits_check_threshold=640)

This will tell you how many bits are used per digit and what the size in bytes is of the underlying C structure. To get the same namedtuple in Python 2, you’d refer to the sys.long_info attribute instead.

While this conversion between fixed- and arbitrary-precision integers is done seamlessly under the hood in Python 3.

Such a representation eliminates integer overflow errors and gives **the illusion of infinite bit-length**, but it requires significantly more memory. Additionally, performing bignum arithmetic is slower than with fixed precision because it can’t run directly in hardware without an intermediate layer of emulation.

**Another challenge is keeping a consistent behavior of the bitwise operators across alternative integer types, which is crucial in handling the sign bit**. Recall that fixed-precision integers in Python use the standard two’s complement representation from C, while **large integers use sign-magnitude**.

To mitigate that difference, Python will do the necessary binary conversion for you. It might change how a number is represented before and after applying a bitwise operator. Here’s a relevant comment from the CPython source code, which explains this in more detail:

Bitwise operations for negative numbers operate as though on a two’s complement representation. So convert arguments from sign-magnitude to two’s complement, and convert the result back to sign-magnitude at the end. (Source)

In other words, **negative numbers are treated as two’s complement bit sequences when you apply the bitwise operators on them, even though the result will be presented to you in sign-magnitude form.** There are ways to emulate the sign bit and some of the unsigned types in Python, though.

### Converting int to Binary
To reveal the bits making up an integer number in Python, you can print a formatted string literal, which optionally lets you specify the number of leading zeros to display:

In [14]:
print(f'{42:b}')  # 42 in binary

print(f'{42:032b}')  # 42 in binary on 32 zero-padded digits

print(bin(42))  # binary literal
print(hex(42))
print(oct(42))

age = 0b101010
print(age)

print(42 == 0b101010 == 0x2a == 0o52)
print(0b101 == 0B101)  # the numeric literals are case insensitive

101010
00000000000000000000000000101010
0b101010
0x2a
0o52
42
True
True


### Converting Binary to int
Calling int() with two arguments will work better in the case of dynamically generated bit strings:

In [17]:
print(int('101010', 2))
print(int('cafe', 16))

42
51966


### Emulating the Sign Bit
When you call bin() on a negative integer, it merely prepends the minus sign to the bit string obtained from the corresponding positive value:

In [18]:
print(bin(-42), bin(42), sep='\n')

-0b101010
0b101010


Changing the sign of a number doesn’t affect the underlying bit string in Python. Conversely, you’re allowed to prefix a bit string with the minus sign when transforming it to decimal form:

In [19]:
int('-101010', 2)

-42

That makes sense in Python because, internally, it doesn’t use the sign bit. You can think of the sign of an integer number in Python as a piece of information stored separately from the modulus.

However, there are a few workarounds that let you emulate fixed-length bit sequences containing the sign bit:

- Bitmask
- **Modulo operation (%)**
- ctypes module
- array module
- struct module
You know from earlier sections that to ensure a certain bit-length of a number, you can use a nifty bitmask. For example, to keep one byte, you can use a mask composed of exactly eight turned-on bits:

In [20]:
# Bit Mask
mask = 0b11111111  # Same as 0xff or 255
bin(-42 & mask)

'0b11010110'

**Masking forces Python to temporarily change the number’s representation from sign-magnitude to two’s complement and then back again.** If you forget about the decimal value of the resulting binary literal, which is equal to 21410, then it’ll represent -4210 in two’s complement. The leftmost bit will be the sign bit.

Alternatively, you can take advantage of the modulo operation that you used previously to simulate the logical right shift in Python:

In [21]:
# Modulo operation
bin(-42 % (1 << 8))  # Give me eight bits

'0b11010110'

If that looks too convoluted for your taste, then you can use one of the modules from the standard library that express the same intent more clearly. For example, using ctypes will have an identical effect:

In [22]:
# ctypes
from ctypes import c_uint8
bin(c_uint8(-42).value)

'0b11010110'

You’ve seen it before, but just as a reminder, it’ll piggyback off the unsigned integer types from C.

Another standard module that you can use for this kind of conversion in Python is the array module. It defines a data structure that’s similar to a list but is only allowed to hold elements of the same numeric type. When declaring an array, you need to indicate its type up front with a corresponding letter:

In [4]:
from array import array
signed = array("b", [-42, 42])
unsigned = array("B")
unsigned.frombytes(signed.tobytes())
print(unsigned)
print(bin(unsigned[0]))
print(bin(unsigned[1]))

array('B', [214, 42])
0b11010110
0b101010


For example, "b" stands for an 8-bit signed byte, while "B" stands for its unsigned equivalent. There are a few other predefined types, such as a signed 16-bit integer or a 32-bit floating-point number.

Copying raw bytes between these two arrays changes how bits are interpreted. However, it takes twice the amount of memory, which is quite wasteful. To perform such a bit rewriting in place, you can rely on the struct module, which uses a similar set of format characters for type declarations:

In [25]:
from struct import pack, unpack
print(unpack("BB", pack("bb", -42, 42)))
print(bin(214))

(214, 42)
0b11010110


Packing lets you lay objects in memory according to the given C data type specifiers. It returns a read-only bytes() object, which contains raw bytes of the resulting block of memory. Later, you can read back those bytes using a different set of type codes to change how they’re translated into Python objects.

Up to this point, you’ve used different techniques to obtain fixed-length bit strings of integers expressed in two’s complement representation. If you want to convert these types of bit sequences back to Python integers instead, then you can try this function:

In [28]:
def from_twos_complement(bit_string, num_bits=32):
    unsigned = int(bit_string, 2)
    sign_mask = 1 << (num_bits - 1)  # For example 0b100000000
    bits_mask = sign_mask - 1        # For example 0b011111111
    return (unsigned & bits_mask) - (unsigned & sign_mask)

The function accepts a string composed of binary digits. First, it converts the digits to a plain unsigned integer, disregarding the sign bit. Next, it uses two bitmasks to extract the sign and magnitude bits, whose locations depend on the specified bit-length. Finally, it combines them using regular arithmetic, knowing that the value associated with the sign bit is negative.

You can try it out against the trusty old bit string from earlier examples:

In [29]:
print(int("11010110", 2))
print(from_twos_complement("11010110"))
print(from_twos_complement("11010110", num_bits=8))

214
214
-42


Python’s int() treats all the bits as the magnitude, so there are no surprises there. However, this new function assumes a 32-bit long string by default, which means the sign bit is implicitly equal to zero for shorter strings. When you request a bit-length that matches your bit string, then you’ll get the expected result.

While integer is the most appropriate data type for working with bitwise operators in most cases, you’ll sometimes need to extract and manipulate fragments of structured binary data, such as image pixels.

In [30]:
bin(127)

'0b1111111'